In [2]:
import pandas as pd
import numpy as np
import datetime as dt

### Identify the new and returning customers

In [10]:
# obtaining the data

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
orders = pd.read_csv('./Data/orders_export.csv')

C:\Users\Angela\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (31,41,44,69,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
orders['Financial Status'].unique()

array(['paid', nan, 'partially_refunded', 'refunded', 'pending'],
      dtype=object)

In [6]:
orders_1 = pd.read_csv('./Data/orders_export_1.csv')
orders_2 = pd.read_csv('./Data/orders_export_2.csv')

In [7]:
orders_com = orders_1.append(orders_2, ignore_index=True)

In [8]:
orders_com['Financial Status'].unique()

array(['paid', nan, 'refunded', 'partially_refunded', 'pending', 'voided'],
      dtype=object)

In [9]:
orders['Order_Date'] = pd.to_datetime(orders['Created at'], format="%Y-%m-%d %H:%M:%S %z", utc=True)
orders['Date'] = pd.to_datetime(orders['Order_Date'].dt.date)
orders['Date_YM'] = orders['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [19]:
orders_com['Order_Date'] = pd.to_datetime(orders_com['Created at'], format="%Y-%m-%d %H:%M:%S %z", utc=True)
orders_com['Date'] = pd.to_datetime(orders_com['Order_Date'].dt.date)
orders_com['Date_YM'] = orders_com['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [77]:
df = orders[['Date','Name','Total', 'Financial Status','Email', 'Subtotal', 'Shipping',
       'Taxes', 'Discount Amount', 'Refunded Amount', 'Billing Country']]
df_com = orders_com[['Date','Name','Total', 'Financial Status','Email', 'Subtotal', 'Shipping',
       'Taxes', 'Discount Amount', 'Refunded Amount', 'Billing Country']]

In [ ]:
df_com.rename(columns={'Date': 'date','Name':'order_id','Total':'total', 'Financial Status':'status','Email':'email', 'Subtotal':'subtotal', 'Shipping':'shipping',
       'Taxes':'taxes', 'Discount Amount':'discount_amount', 'Refunded Amount':'refunded_amount', 'Billing Country': 'billing_country'}, inplace=True)

In [ ]:
df.rename(columns={'Date': 'date','Name':'order_id','Total':'total', 'Financial Status':'status','Email':'email', 'Subtotal':'subtotal', 'Shipping':'shipping',
       'Taxes':'taxes', 'Discount Amount':'discount_amount', 'Refunded Amount':'refunded_amount', 'Billing Country': 'billing_country'}, inplace=True)

In [84]:
df_com.shape, df.shape

((21906, 17), (4268, 17))

In [2]:
# Customers

In [31]:
%run ./cohorts_pipeline_koi.py

In [82]:
df_com = first_order(df_com)
df = first_order(df)

In [83]:
df_com.shape, df.shape

((21906, 17), (4268, 17))

In [86]:
df_com.head(2)

,email,date,order_id,total,status,subtotal,shipping,taxes,discount_amount,refunded_amount,billing_country,order_date,first_order,first_order_ym,date_ym,customer_type,days_between
0,ernst.hiemstra@gmail.com,2018-08-09,KOI1002,84.97,paid,84.97,0.0,14.75,0.0,0.0,NL,2018-08-09,2018-08-09,2018-08,2018-08,New,0.0
1,andre@kingsofindigo.com,2018-08-14,KOI1003,184.92,paid,184.92,0.0,32.09,0.0,0.0,NL,2018-08-14,2018-08-14,2018-08,2018-08,New,0.0


In [87]:
df_com.isnull().sum()

email              12
date                0
order_id            0
total               0
status              0
subtotal            0
shipping            0
taxes               0
discount_amount     0
refunded_amount     0
billing_country    86
order_date          0
first_order        12
first_order_ym     12
date_ym             0
customer_type       0
days_between        0
dtype: int64

In [89]:
df.head(2)

,email,date,order_id,total,status,subtotal,shipping,taxes,discount_amount,refunded_amount,billing_country,order_date,first_order,first_order_ym,date_ym,customer_type,days_between
0,andrekortekaas@hotmail.com,2019-12-04,20001001,8.20,paid,3.25,4.95,0.56,29.24,0.0,NL,2019-12-04,2019-12-04,2019-12,2019-12,New,0.0
1,andrekortekaas@hotmail.com,2019-12-10,20001002,24.95,paid,20.00,4.95,3.47,179.95,0.0,NL,2019-12-10,2019-12-04,2019-12,2019-12,Returning,6.0


In [90]:
df.isnull().sum()

email              0
date               0
order_id           0
total              0
status             0
subtotal           0
shipping           0
taxes              0
discount_amount    0
refunded_amount    0
billing_country    0
order_date         0
first_order        0
first_order_ym     0
date_ym            0
customer_type      0
days_between       0
dtype: int64

In [91]:
df_com[df_com['date']> '2020-01-01']['customer_type'].value_counts()

New          7260
Returning    4098
Name: customer_type, dtype: int64

In [92]:
df[df['date']> '2020-01-01']['customer_type'].value_counts()

New          3543
Returning     628
Name: customer_type, dtype: int64

In [95]:
df_com[df_com['date']> '2021-02-01']['subtotal'].sum() + df[df['date']> '2021-02-01']['subtotal'].sum()

190115.4

In [101]:
summary_de = df.groupby(['date_ym', 'billing_country', 'customer_type']).agg({'subtotal':pd.Series.sum, 'order_id':pd.Series.count, 'shipping':pd.Series.sum, 'taxes':pd.Series.sum, 'total':pd.Series.sum, 'discount_amount':pd.Series.sum, 'refunded_amount':pd.Series.sum}).reset_index()
summary_com = df_com.groupby(['date_ym', 'billing_country', 'customer_type']).agg({'subtotal':pd.Series.sum, 'order_id':pd.Series.count, 'shipping':pd.Series.sum, 'taxes':pd.Series.sum, 'total':pd.Series.sum, 'discount_amount':pd.Series.sum, 'refunded_amount':pd.Series.sum}).reset_index()

In [114]:
summary_com[summary_com['date_ym']== '2021-02'][summary_com['billing_country'].isin(['DE', 'AT', 'CH'])]['subtotal'].sum() + summary_de[summary_de['date_ym']== '2021-02'][summary_de['billing_country'].isin(['DE', 'AT', 'CH'])]['subtotal'].sum()

<ipython-input-114-f72d2f490e54>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  summary_com[summary_com['date_ym']== '2021-02'][summary_com['billing_country'].isin(['DE', 'AT', 'CH'])]['subtotal'].sum() + summary_de[summary_de['date_ym']== '2021-02'][summary_de['billing_country'].isin(['DE', 'AT', 'CH'])]['subtotal'].sum()


77939.53

In [115]:
78159.51 - 77939.53

219.97999999999593

In [118]:
summary_de['TO_EXSHIP_EXVAT'] = summary_de['subtotal'] - summary_de['taxes']
summary_com['TO_EXSHIP_EXVAT'] = summary_com['subtotal'] - summary_com['taxes']

In [119]:
summary_de['TO_EXSHIP_EXVAT_EXRET'] = summary_de['TO_EXSHIP_EXVAT'] - summary_de['refunded_amount']
summary_com['TO_EXSHIP_EXVAT_EXRET'] = summary_com['TO_EXSHIP_EXVAT'] - summary_com['refunded_amount']

In [120]:
summary_de.to_clipboard(decimal=',', index=False)

In [121]:
summary_com.to_clipboard(decimal=',', index=False)

### Find the return rate at brand level

In [125]:
# .de
# just for the DACH countries
refund_TX = orders[orders['Date'] >= '2020-01-01'][orders['Date']<'2021-01-01'][orders['Billing Country'].isin(['DE', 'AT', 'CH'])][orders['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
all_TX = orders[orders['Date'] >= '2020-01-01'][orders['Date']<'2021-01-01'][orders['Billing Country'].isin(['DE', 'AT', 'CH'])][orders['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()
refund_TX / all_TX

<ipython-input-125-9147afbf8767>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  refund_TX = orders[orders['Date'] >= '2020-01-01'][orders['Date']<'2021-01-01'][orders['Billing Country'].isin(['DE', 'AT', 'CH'])][orders['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
<ipython-input-125-9147afbf8767>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_TX = orders[orders['Date'] >= '2020-01-01'][orders['Date']<'2021-01-01'][orders['Billing Country'].isin(['DE', 'AT', 'CH'])][orders['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()


0.415167865707434

In [128]:
# .de
# all countries
refund_TX = orders[orders['Date'] >= '2020-01-01'][orders['Date']<'2021-01-01'][orders['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
all_TX = orders[orders['Date'] >= '2020-01-01'][orders['Date']<'2021-01-01'][orders['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()
refund_TX / all_TX

<ipython-input-128-9ebe732b2d8b>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  refund_TX = orders[orders['Date'] >= '2020-01-01'][orders['Date']<'2021-01-01'][orders['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
<ipython-input-128-9ebe732b2d8b>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_TX = orders[orders['Date'] >= '2020-01-01'][orders['Date']<'2021-01-01'][orders['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()


0.41438763376932225

In [126]:
# .com
# all countries
refund_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
all_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()
refund_TX_com / all_TX_com

<ipython-input-126-0417e2c09336>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  refund_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
<ipython-input-126-0417e2c09336>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()


0.3614678899082569

Apparently on the .com site, the percentage of refund transactions is 36.14%

In [127]:
# .com
# just for the DACH countries
refund_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Billing Country'].isin(['DE', 'AT', 'CH'])][orders_com['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
all_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Billing Country'].isin(['DE', 'AT', 'CH'])][orders_com['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()
refund_TX_com / all_TX_com

<ipython-input-127-5338fe64266a>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  refund_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Billing Country'].isin(['DE', 'AT', 'CH'])][orders_com['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
<ipython-input-127-5338fe64266a>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Billing Country'].isin(['DE', 'AT', 'CH'])][orders_com['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()


0.34392265193370164

In [122]:
refund_total = summary_de[summary_de['date_ym']>= '2020-01'][summary_de['date_ym'] < '2021-01']['refunded_amount'].sum() 
TO = summary_de[summary_de['date_ym']>= '2020-01'][summary_de['date_ym'] < '2021-01']['TO_EXSHIP_EXVAT'].sum()
refund_total / TO 

<ipython-input-122-ca198f55eb54>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  refund_total = summary_de[summary_de['date_ym']>= '2020-01'][summary_de['date_ym'] < '2021-01']['refunded_amount'].sum()
<ipython-input-122-ca198f55eb54>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  TO = summary_de[summary_de['date_ym']>= '2020-01'][summary_de['date_ym'] < '2021-01']['TO_EXSHIP_EXVAT'].sum()


0.5010962064970838

In [129]:
refund_total_com = summary_com[summary_com['date_ym']>= '2020-01'][summary_com['date_ym'] < '2021-01']['refunded_amount'].sum() 
TO_com = summary_com[summary_com['date_ym']>= '2020-01'][summary_com['date_ym'] < '2021-01']['TO_EXSHIP_EXVAT'].sum()
refund_total_com / TO_com

<ipython-input-129-4b1f95da0fe6>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  refund_total_com = summary_com[summary_com['date_ym']>= '2020-01'][summary_com['date_ym'] < '2021-01']['refunded_amount'].sum()
<ipython-input-129-4b1f95da0fe6>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  TO_com = summary_com[summary_com['date_ym']>= '2020-01'][summary_com['date_ym'] < '2021-01']['TO_EXSHIP_EXVAT'].sum()


0.42205277837195265